In [1]:
import mne
import mne_bids
import numpy as np
import autoreject as ar
import os

%matplotlib qt

In [2]:
bids_root = "/Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS"
output_dir = "/Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/output"
subjects_dir = "/Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/fs_subjects"

In [56]:
for subject in ["ap", "ca", "ha", "ja", "mv", "wc", "zt"]:
    
    epochs_list = []

    for task in ["clean", "noisy"]:

        path = mne_bids.BIDSPath(root=bids_root, subject=subject, session="01", task=task, datatype="ieeg", suffix="ieeg")
        raw = mne_bids.read_raw_bids(path, extra_params=dict(preload=True))

        montage = raw.get_montage()

        # add fiducials to montage
        montage.add_estimated_fiducials(f"{subjects_dir}/{subject}")

        #trans = mne_bids.get_head_mri_trans(bids_path=path, fs_subject=subject,
        #                                    fs_subjects_dir=subjects_dir)

        # now with fiducials assigned, the montage will be properly converted
        # to "head" which is what MNE requires internally (this is the coordinate
        # system with the origin between LPA and RPA whereas MNI has the origin
        # at the posterior commissure)
        raw.set_montage(montage)

        raw.filter(0.15, None, n_jobs=-1)
        raw.notch_filter(range(60, 500, 60), method="spectrum_fit", filter_length="5s", n_jobs=-1)

        events, event_id = mne.events_from_annotations(raw)

        epochs = mne.Epochs(raw, events, event_id, baseline=None, detrend=0, tmin=-0.2, tmax=0.8, preload=True)

        epochs_filt = epochs.copy().filter(1, None)
        reject = ar.get_rejection_threshold(epochs_filt, ch_types="ecog", verbose=True)
        print(reject)
        epochs_filt.drop_bad(reject=reject)

        ica = mne.preprocessing.ICA(n_components=0.95, max_iter="auto")
        ica.fit(epochs_filt)
        epochs_clean = ica.apply(epochs.copy())

        #reject = ar.get_rejection_threshold(epochs_clean, ch_types="ecog", verbose=True)
        #print(reject)
        #epochs_clean.drop_bad(reject=reject)

        epochs_clean.apply_baseline((-0.2, 0))

        epochs_list.append(epochs_clean)

    epochs = mne.concatenate_epochs(epochs_list, add_offset=100000)
    
    epochs.save(f"{output_dir}/epoched/{subject}-epo.fif.gz", overwrite=True)

Extracting parameters from /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS/sub-ap/ses-01/ieeg/sub-ap_ses-01_task-clean_ieeg.vhdr...
Setting channel info structure...
Reading 0 ... 268399  =      0.000 ...   268.399 secs...
Reading events from /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS/sub-ap/ses-01/ieeg/sub-ap_ses-01_task-clean_events.tsv.
Reading channel info from /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS/sub-ap/ses-01/ieeg/sub-ap_ses-01_task-clean_channels.tsv.
Reading electrode coords from /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS/sub-ap/ses-01/ieeg/sub-ap_ses-01_space-ACPC_electrodes.tsv.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.15 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done  41 out of  41 | elapsed:    1.8s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    8.6s
[Parallel(n_jobs=4)]: Done  41 out of  41 | elapsed:   21.2s finished


Removed notch frequencies (Hz):
     60.00 : 4346 windows
    120.00 : 4346 windows
    180.00 : 4346 windows
    239.00 :   41 windows
    240.00 : 4346 windows
    241.00 :   41 windows
    299.00 : 4346 windows
    300.00 : 4346 windows
    301.00 : 4346 windows
    359.00 : 4346 windows
    360.00 : 4346 windows
    361.00 : 4346 windows
    419.00 : 4346 windows
    420.00 : 4346 windows
    421.00 : 4346 windows
    479.00 : 4346 windows
    480.00 : 4346 windows
    481.00 : 4346 windows
Used Annotations descriptions: ['face/clean', 'house/clean']
Not setting metadata
300 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 300 events and 1001 original time points ...
0 bad epochs dropped
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband rippl

/var/folders/_r/kcfrqwd143bfg9vx3jtb4l6w0000gs/T/ipykernel_46056/392848324.py:31: RuntimeWarning: filter_length (3301) is longer than the signal (1001), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_filt = epochs.copy().filter(1, None)


Estimating rejection dictionary for ecog
{'ecog': 0.07533395920228132}
0 bad epochs dropped
Fitting ICA to data using 41 channels (please be patient, this may take a while)
Selecting by explained variance: 30 components
Fitting ICA took 21.8s.
Applying ICA to Epochs instance
    Transforming to ICA space (30 components)
    Zeroing out 0 ICA components
    Projecting back using 41 PCA components
Applying baseline correction (mode: mean)
Extracting parameters from /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS/sub-ap/ses-01/ieeg/sub-ap_ses-01_task-noisy_ieeg.vhdr...
Setting channel info structure...
Reading 0 ... 666239  =      0.000 ...   666.239 secs...
Reading events from /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS/sub-ap/ses-01/ieeg/sub-ap_ses-01_task-noisy_events.tsv.
Reading channel info from /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS/sub-ap/ses-01/ieeg/sub-ap_ses-01_task-noisy_channels.tsv.
Readi

...

Removed notch frequencies (Hz):
     60.00 : 15486 windows
    120.00 : 15486 windows
    180.00 : 15486 windows
    240.00 : 15486 windows
    299.00 : 15486 windows
    300.00 : 15486 windows
    301.00 : 15486 windows
    359.00 : 15486 windows
    360.00 : 15486 windows
    361.00 : 15486 windows
    419.00 : 15486 windows
    420.00 : 15486 windows
    421.00 : 15486 windows
    479.00 : 15486 windows
    480.00 : 15486 windows
    481.00 : 15486 windows
Used Annotations descriptions: ['face/noisy', 'house/noisy']
Not setting metadata
630 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 630 events and 1001 original time points ...
0 bad epochs dropped
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- L

/var/folders/_r/kcfrqwd143bfg9vx3jtb4l6w0000gs/T/ipykernel_46056/392848324.py:31: RuntimeWarning: filter_length (3301) is longer than the signal (1001), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_filt = epochs.copy().filter(1, None)


Estimating rejection dictionary for ecog
{'ecog': 0.05530791456114876}
0 bad epochs dropped
Fitting ICA to data using 58 channels (please be patient, this may take a while)
Selecting by explained variance: 34 components
Fitting ICA took 42.0s.
Applying ICA to Epochs instance
    Transforming to ICA space (34 components)
    Zeroing out 0 ICA components
    Projecting back using 58 PCA components
Applying baseline correction (mode: mean)


/var/folders/_r/kcfrqwd143bfg9vx3jtb4l6w0000gs/T/ipykernel_46056/392848324.py:48: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs(epochs_list, add_offset=100000)


Not setting metadata
930 matching events found
Applying baseline correction (mode: mean)
0 bad epochs dropped


In [54]:
mne.viz.plot_epochs_image(epochs.apply_baseline((-0.2,0)), picks="all", combine="gfp")

Applying baseline correction (mode: mean)
Not setting metadata
930 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
combining channels using "gfp"


[<Figure size 1280x960 with 3 Axes>]